In [1]:
##############################################################################
### Examples to navigate Open Asset Pricing data:                          ###
### Calculate average returns of investment  predictors and NYSE porfolios ###
##############################################################################

In [2]:
# Install the Open Asset Pricing library
%pip install openassetpricing

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import sqlalchemy as sa
import wrds
 
print(pd.__version__)
print(sa.__version__)
print(wrds.__version__)

2.2.2
2.0.32
3.2.0


In [3]:
# Set up environment
import pandas as pd
import numpy as np
import openassetpricing as oap

# Initialize OpenAP using the 2024 data release 
openap = oap.OpenAP("2024")

In [4]:
# View available datasets
openap.list_datasets()

┌─────────────────────────────────────────────────┬─────────────────────┐
│ CZ data file                                    │ Name for download   │
├─────────────────────────────────────────────────┼─────────────────────┤
│ signed_predictors_dl_wide.zip                   │ char_predictors     │
│ PredictorAltPorts_Deciles.zip                   │ port_deciles_ew     │
│ PredictorAltPorts_DecilesVW.zip                 │ port_deciles_vw     │
│ PredictorAltPorts_LiqScreen_ME_gt_NYSE20pct.zip │ port_ex_nyse_p20_me │
│ PredictorAltPorts_LiqScreen_NYSEonly.zip        │ port_nyse           │
│ PredictorAltPorts_LiqScreen_Price_gt_5.zip      │ port_ex_price5      │
│ PredictorAltPorts_Quintiles.zip                 │ port_quintiles_ew   │
│ PredictorAltPorts_QuintilesVW.zip               │ port_quintiles_vw   │
│ PredictorPortsFull.csv                          │ port_op             │
│ SignalDoc.csv                                   │ signal_doc          │
└─────────────────────────────────────

In [5]:
# Download original (OP) version of the portfolios
df = openap.dl('port_op', 'pandas')
df.head()


Data is downloaded: 2 mins


,signalname,port,date,ret,signallag,Nlong,Nshort
0,AM,01,1951-07-31,7.661648,0.681619,67,0
1,AM,01,1951-08-31,4.273654,0.639816,67,0
2,AM,01,1951-09-28,1.315270,0.617079,67,0
3,AM,01,1951-10-31,-3.942987,0.612266,67,0
4,AM,01,1951-11-30,1.028675,0.637710,67,0


In [6]:
# 1. Check how investment predictors have performed in past 10 years: 
# 1.1 Use signal documentation to find investment predictors
df1 = openap.dl('signal_doc', 'pandas')

# Filter for "Predictor" in signals category
df_predictor = df1[(df1["Cat.Signal"] == "Predictor")]

# Look for word "investment" in "LongDescription"
df_predictor["LongDescription"] = df_predictor["LongDescription"].str.lower()
search_term = "investment"
bool_vector = pd.Series(df_predictor["LongDescription"]).str.contains(search_term).tolist()
true_indices2 = np.where(bool_vector)[0]

# Keep only predictor signals related to investment
df_predictor = df_predictor.iloc[true_indices2]

# Extract names of investment predictors
invest_pred_names = df_predictor["Acronym"].tolist()

# Describe characteristics of investment predictors
invest_pred = df_predictor[["Acronym", "Authors", "Year", "Journal", "LongDescription"]].copy()
invest_pred


Data is downloaded: 3s


C:\Users\test\AppData\Local\Temp\ipykernel_20404\3224976927.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictor["LongDescription"] = df_predictor["LongDescription"].str.lower()


,Acronym,Authors,Year,Journal,LongDescription
46,DelLTI,Richardson et al.,2005,JAE,change in long-term investment
89,Investment,"Titman, Wei and Xie",2004,JFQA,investment to revenue
176,BrandInvest,"Belo, Lin and Vitorino",2014,RED,brand capital investment


In [20]:
# 1.2 Filter for investment predictors in returns dataset
df2 = openap.dl('port_op', 'pandas')
filtered_df2 = df2[df2["signalname"].isin(invest_pred_names)]

# Filter for last 10 years
filtered_df2["date"] = pd.to_datetime(filtered_df2["date"])
filter_date = '2014-01-01'
filtered_df2 = filtered_df2[filtered_df2["date"] >= filter_date]

# Calculate average returns by signal
invest_pred_returns = filtered_df2.groupby('signalname')["ret"].mean()
print(invest_pred_returns)


Data is downloaded: 18s
signalname
BrandInvest    0.521986
DelLTI         0.552153
Investment     0.876242
Name: ret, dtype: float64


C:\Users\test\AppData\Local\Temp\ipykernel_16160\751025204.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df2["date"] = pd.to_datetime(filtered_df2["date"])


In [21]:
# 2. Calculate portfolio performance under a specific liquidy screen
# Open New York Stock Exchange (NYSE) portfolios dataset
df3 = openap.dl('port_nyse', 'pandas')

# Calculate average returns by signal
nyse_mean_returns = df3.groupby('signalname')["ret"].mean()
print(nyse_mean_returns)


Data is downloaded: 8s
signalname
AM                  1.005376
AOP                 0.964989
AbnormalAccruals    1.067322
Accruals            1.071110
AccrualsBM          1.060262
                      ...   
std_turn            1.158646
tang                1.010259
zerotrade12M        1.126181
zerotrade1M         1.109966
zerotrade6M         1.121755
Name: ret, Length: 212, dtype: float64
